<a href="https://colab.research.google.com/github/Adhi1904/movieSentimental_analysis/blob/main/movieReview_sentimentalAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import nltk

In [ ]:
from google.colab import files
import io
data=files.upload()

Saving movieReviews.csv to movieReviews.csv


In [ ]:
df=pd.read_csv(io.StringIO(data['movieReviews.csv'].decode('utf-8')))

In [ ]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
df.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [ ]:
df.isna().sum()

review       0
sentiment    0
dtype: int64

In [ ]:
for i in range(0,len(df["sentiment"])):
    if(df['sentiment'][i]=='positive'):
        df['sentiment'][i]=1
    else:
        df['sentiment'][i]=0

In [ ]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [ ]:
df['sentiment'].value_counts()

1    25000
0    25000
Name: sentiment, dtype: int64

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import re
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

def clean_text(Review):
    Review = str(Review).lower() # convert to lowercase
    Review = re.sub('\[.*?\]', '', Review) 
    Review = re.sub('https?://\S+|www\.\S+', '', Review) # Remove URls
    Review = re.sub('<.*?>+', '', Review)
    Review = re.sub(r'[^a-z0-9\s]', '', Review) # Remove punctuation
    Review = re.sub('\n', '', Review)
    Review = re.sub('\w*\d\w*', '', Review)
    Review = [lemmatizer.lemmatize(word) for word in Review.split() if not word in set(stopwords.words('english'))]
    Review = ' '.join(Review)
    return Review

In [ ]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
df['review'] = df['review'].apply(clean_text)

In [ ]:
df.head()

,review,sentiment
0,one reviewer mentioned watching oz episode you...,1
1,wonderful little production filming technique ...,1
2,thought wonderful way spend time hot summer we...,1
3,basically family little boy jake think zombie ...,0
4,petter matteis love time money visually stunni...,1


In [ ]:
from sklearn.model_selection import train_test_split
import string, nltk
from nltk import word_tokenize
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Bidirectional, Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
x = df['review']
y = df['sentiment']


In [ ]:
y

0        1
1        1
2        1
3        0
4        1
        ..
49995    1
49996    0
49997    0
49998    0
49999    0
Name: sentiment, Length: 50000, dtype: object

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(np.array(x), np.array(y), test_size=0.3,random_state=42)

In [ ]:
y_test

array([1, 1, 0, ..., 0, 1, 1], dtype=object)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(use_idf=True, tokenizer=word_tokenize)
x_train_tf = tfidf.fit_transform(x_train)
x_test_tf = tfidf.transform(x_test)

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(type(x_train))
y_train = np.array([np.array(y) for y in y_train])
y_test = np.array([np.array(y) for y in y_test])

(35000,)
(15000,)
<class 'numpy.ndarray'>


In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)
x_train_seq = tokenizer.texts_to_sequences(x_train)
x_test_seq = tokenizer.texts_to_sequences(x_test)


In [ ]:
x_train_seq_padded = pad_sequences(x_train_seq, maxlen=64)
x_test_seq_padded = pad_sequences(x_test_seq, maxlen=64)
x_train_seq_padded[0]
print(type(x_test_seq_padded))

<class 'numpy.ndarray'>


In [ ]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(len(tokenizer.index_word)+1,64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='tanh'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          10229632  
                                                                 
 bidirectional (Bidirectiona  (None, 64)               24832     
 l)                                                              
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 10,258,689
Trainable params: 10,258,689
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor="val_loss",patience=3,verbose=True)

In [ ]:
history = model.fit(x_train_seq_padded, y_train, epochs=10,validation_data=(x_test_seq_padded, y_test),callbacks=[early_stop])

Epoch 1/10
1094/1094 [==============================] - 164s 147ms/step - loss: 0.3687 - accuracy: 0.8327 - val_loss: 0.3070 - val_accuracy: 0.8656
Epoch 2/10
1094/1094 [==============================] - 157s 143ms/step - loss: 0.1466 - accuracy: 0.9449 - val_loss: 0.3534 - val_accuracy: 0.8493
Epoch 3/10
1094/1094 [==============================] - 157s 144ms/step - loss: 0.0410 - accuracy: 0.9860 - val_loss: 0.5405 - val_accuracy: 0.8445
Epoch 4/10
1094/1094 [==============================] - 158s 145ms/step - loss: 0.0154 - accuracy: 0.9947 - val_loss: 0.7485 - val_accuracy: 0.8431
Epoch 4: early stopping


In [ ]:
from sklearn.metrics import roc_auc_score
pred_train = model.predict(x_train_seq_padded)
pred_test = model.predict(x_test_seq_padded)
print('Training Score: ' + str(roc_auc_score(y_train, pred_train)))
print('Testing Score: ' + str(roc_auc_score(y_test, pred_test)))

Training Score: 0.9999728972581852
Testing Score: 0.9190795027331761


In [ ]:
y_test

array([1, 1, 0, ..., 0, 1, 1])

In [ ]:
pred_test = np.where(pred_test > 0.5, 1, 0)

In [ ]:
pred_test

array([[0],
       [1],
       [0],
       ...,
       [0],
       [1],
       [1]])

In [ ]:
from sklearn.metrics import accuracy_score
print("Accuracy Score")
print(accuracy_score(pred_test, y_test))

Accuracy Score
0.8430666666666666


In [ ]:
print(pred_test[0:24].reshape( 1, -1))
print(y_test[0:24])

[[0 1 0 1 0 0 1 0 1 0 0 1 0 0 0 1 1 1 1 1 0 1 1 0]]
[1 1 0 1 0 1 1 1 0 0 0 1 0 0 0 1 1 1 1 1 1 1 1 1]
